In [ ]:
from IPython.core.display import HTML
with open("style/notebook.css", "r") as f:
    s = f"<style>{f.read()}</style>"
HTML(s)

In [ ]:
from bokeh.plotting import figure, ColumnDataSource
from bokeh.io import show, output_notebook, push_notebook
from bokeh.layouts import row
import subprocess
import swarming
import numpy as np
import pandas as pd

from dask.distributed import Client, LocalCluster, progress
import dask.bag
import dask.dataframe

output_notebook()

<div class="title">

<h1 class="title">Predicting the behavior of swarms</h1>
 
Modelling interacting particles
    
*by Andreas Roth*

**2021-08-23**
</div>

<div class="sect">
    
## The model
    
* How do swarms of birds or fish align themselves?
* Who decides about collective movement?
    
J. Carrillo, A. Klar, S. Martin, and S. Tiwari, [Self-propelled interacting particle systems with
roosting force](https://www.worldscientific.com/doi/abs/10.1142/S0218202510004684). *Mathematical Models and Methods in Applied Sciences, 20(supp01), 2010*

Carrillo, J.A.; Klar, A.; Roth, A., [Single to double mill small noise transition via semi-lagrangian finite volume methods](https://www.intlpress.com/site/pub/pages/journals/items/cms/content/vols/0014/0004/a012/), *Communications in mathematical sciences 14 (2016), No.4, pp.1111-1136, ISSN: 1539-6746*

</div>

## Basic ideas

<div class="row">
<div class="column">
<img src="img/velocity.png">  
</div>
<div class="column">

We look at the behavior of $n$ point masses
* position $x_i$ and velocity $v_i$, where $i = 1,...,n$

</div>
</div>

## Basic ideas

<div class="row">
<div class="column">
<img src="img/repulsion.png">  
</div>
<div class="column">

We look at the behavior of $n$ particles
* position $x_i$ and velocity $v_i$, where $i = 1,...,n$
* interaction force between pairs of particles depends on distance
    * if to close: **repulsion**
    
</div>
</div>

## Basic ideas

<div class="row">
<div class="column">
<img src="img/attraction.png">  
</div>
<div class="column">

We look at the behavior of $n$ particles
* position $x_i$ and velocity $v_i$, where $i = 1,...,n$
* interaction force between pairs of particles depends on distance
    * if to close: **repulsion**
    * if to far: **attraction**
* forces balance each other
* pairwise interactions on one particle **superpose** each other
* evolution over time follows **Newton's second law**
</div>
</div>

## Model equations

System of ordinary differential equations:
$$
\begin{align}
dx_i =& v_i \cdot dt\\
dv_i =& v_i(\alpha-\beta|v_i|) \\
&- \frac{1}{n}\sum_{i\neq j}F(|r_{ij}|) \frac{r_{ij}}{|r_{ij}|}
\end{align}
$$

* $2n$ equations
* $r_{ij} = x_i-x_j$ connection vector between particles $i$ and $j$
* Computation effort goes with $n^2$ due to summing pairwise forces!

### Force term

One possibility to achieve what we want is

$$
F(r) = \frac{C_a}{l_a}\cdot \exp(-\frac{r}{l_a})-\frac{C_r}{l_r}\cdot \exp(-\frac{r}{l_r})
$$

In [ ]:
r = np.linspace(7, 200, 200)
f = figure(title="Force over distance", plot_width=1500, plot_height=200)
f.line(x=r, y=swarming.force_of_distance(r), line_width=3)
show(f)

<div class="sect">
  
## How to compute
    
* all code is published under MIT License at https://github.com/scherbertlemon/swarming
* we make use of the ``swarming`` python package
* [Bokeh](https://bokeh.org) is used for all visualizations
* [Dask](https://dask.org) is used for parallel computing
    
</div>

## Generating some particles

In [ ]:
swarm = swarming.InitialCondition(condition="circular", n_particles=300)
show(row(swarm.plot(plot_width=600, plot_height=600), swarm.plot_density(plot_width=800, plot_height=600, size=3)))

## Letting the particles move

In [ ]:
nh = show(swarm.plot(plot_width=600, plot_height=600), notebook_handle=True)

In [ ]:
swarm.evolve(0.1, 10).update_cds()
push_notebook(handle=nh)

### More interactive

In [ ]:
proc = subprocess.Popen(["bokeh", "serve", "--port", "5006", "../bokeh_app/swarm.py"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

More interactively with [Bokeh app](http://localhost:5006)!
* Javascript frontend connected to a python backend
* Python runs the model
* Sliders change parameters while model is running

In [ ]:
import psutil
parentproc = psutil.Process(proc.pid)
for subproc in parentproc.children(recursive=True):
    subproc.kill()
parentproc.kill()

## Without interaction

Apparently, there is only a finite set of "equilibria" for this system, depending on the initial condition.

### All velocities aligned

In [ ]:
swarm.n = 400
show(row(swarm.set_initial("square").plot_density(size=5),
        swarm.set_initial("square").record_for_time(120, 0.5, 2, lr=1).plot_density(size=3)))

## Without interaction

Apparently, there is only a finite set of "equilibria" for this system, depending on the initial condition.

### Random velocities

In [ ]:
show(row(swarm.set_initial("randomspeed").plot_density(size=5),
        swarm.set_initial("randomspeed").record_for_time(120, 0.5, 2, lr=1).plot_density(size=2)))

<div class="sect">
    
## Parameter study
    
* To check if the "donut" equilibrium always occurs for a given initial condition (random velocity), we run a parameter study
* We need to sample model parameters randomly and cover the complete parameter space: **Latin Hypercube Sampling**
* We use Dask to run the models for each parameter sample in parallel
    
</div>

## Dask cluster and client

In [ ]:
cluster = LocalCluster(n_workers=4)
client = Client(cluster)
cluster

* There are backends to run Dask cluster on all kinds of systems (IBM LSF, Hadoop, ...)
* We run a ``LocalCluster`` on one CPU
* Dask client connects to cluster over the network (really distributed computing!)
* [Cluster dashboard](http://localhost:8787) shows cluster load etc.

### Parameter sampling

Model has parameters attraction strength $C_a$, attraction range $l_a$, repulsion strength $C_r$, repulsion range $l_r$.

In [ ]:
params = swarming.Parameters([("la", 100, 40, 160), ("lr", 2, 0.5, 3.5), ("Cr", 30, 5, 35)])
params

In [ ]:
pd.DataFrame(params.sampling_dicts(n_samples=10))

## Run and store model for every of those parameter sets

We build a Dask *task graph*. No calculation happens here.

In [ ]:
def calc(dct):
    return (dct, swarming.InitialCondition(condition="randomspeed", n_particles=400).record_for_time(100, 0.5, 2, **dct).history.to_dict(orient="records"))
def explode(params, result):
    return [{**params, **r} for r in result]

dag = dask.bag.from_sequence(params.sampling_dicts(n_samples=24)) \
    .map(calc) \
    .starmap(explode) \
    .flatten() \
    .to_dataframe() \
    .to_parquet("results.parq", engine="pyarrow", overwrite=True, compute=False)

### Visualizing and computing the task graph

In [ ]:
dag.visualize()

In [ ]:
future = client.compute(dag)  # send the graph to the workers for computing
# progress(future)  # does not display correctly in slideshow

### Evaluate stored results

We have written **all particle positions / velocities** at defined **time steps** with the **used parameter sample**:

In [ ]:
data = dask.dataframe.read_parquet("results.parq")
data.head(150, npartitions=2)

### Evaluate stored results

* We want to plot the end densities, so select records with the maximum computed time
* Note, that at no time the dataset is completely in memory!
* When calling ``compute`` on a DAG, the result is actually transferred into local memory as ``pandas.DataFrame``

In [ ]:
data.groupby(params.names).agg({"time": "max"}).compute().head(3)

In [ ]:
final_states = data.loc[abs(data["time"]-100.0)<1e-12, :].compute()
final_states.head(3)

## Density plots of final states

Apparently, this parameterset delivers donuts in varying shapes.

In [ ]:
show(swarming.get_density_plots(final_states, param_names=params.names, ncols=4, size=3))

### Tidy up

We close the Dask cluster and client:

In [ ]:
client.close()
cluster.close()

<div class="title">
    
<h1 class="title">Thanks for your attention</h1>
    
Time for your questions!
    
</div>    